# 🤖 Agent Development - Risk Analyst & Compliance Officer

Welcome to Phase 2 & 3 of the project! In this notebook, you'll develop and test both AI agents:

1. **Risk Analyst Agent** (Chain-of-Thought prompting)
2. **Compliance Officer Agent** (ReACT prompting)

## 🎯 Learning Objectives
- Implement Chain-of-Thought prompting for systematic reasoning
- Build ReACT prompting for structured action-taking
- Handle structured JSON outputs from LLMs
- Create robust error handling and validation
- Test agents with real financial data

## 🚀 Setup and Environment

In [1]:
# Import required libraries
import os
import sys
import json
from dotenv import load_dotenv
from datetime import datetime

# Add src directory to Python path for module imports
sys.path.append(os.path.abspath('../src'))

# Load environment variables
load_dotenv('../.env')

print("📚 Libraries loaded!")
print("🔐 Environment variables loaded")
print("📂 Source directory added to Python path:", os.path.abspath('../src'))

📚 Libraries loaded!
🔐 Environment variables loaded
📂 Source directory added to Python path: c:\Projects\Agentic AI For Financial Services\1. Prompting for LLM Reasoning and Planning for Financial Services\Project - Automated SAR Detection & Reporting System\starter\src


In [2]:
import openai

# Option 2: Manual setup (for early development)
openai_api_key = os.getenv('OPENAI_VOCAREUM_API_KEY')

if not openai_api_key:
    print("⚠️ WARNING: No OpenAI API key found!")
    print("Please set OPENAI_VOCAREUM_API_KEY in your .env file")
    print("Get your Vocareum OpenAI API key from 'Cloud Resources' in your workspace")
else:
    # Vocareum requires routing through their servers
    client = openai.OpenAI(
        base_url="https://openai.vocareum.com/v1",
        api_key=openai_api_key
    )
    print("✅ OpenAI client initialized with Vocareum routing")
    print(f"🔑 API key: {openai_api_key[:8]}...{openai_api_key[-4:]}")
    print("📍 Base URL: https://openai.vocareum.com/v1")
    print("\n💡 Tip: After implementing foundation_sar.py, you can use:")
    print("   from src import create_vocareum_openai_client")
    print("   client = create_vocareum_openai_client()")

✅ OpenAI client initialized with Vocareum routing
🔑 API key: voc-1769...4816
📍 Base URL: https://openai.vocareum.com/v1

💡 Tip: After implementing foundation_sar.py, you can use:
   from src import create_vocareum_openai_client
   client = create_vocareum_openai_client()


## 📊 Phase 1 Review: Load Foundation Components

Before building agents, let's ensure your foundation components are working.

In [3]:
# TODO: Import your implemented foundation components
# Uncomment and modify these imports once you've implemented foundation_sar.py

from foundation_sar import (
     CustomerData,
     AccountData,
     TransactionData,
     CaseData,
     RiskAnalystOutput,
     ComplianceOfficerOutput,
     ExplainabilityLogger,
     DataLoader,
     load_csv_data,
     normalize_transaction_dict
)
from risk_analyst_agent import RiskAnalystAgent

print("📋 TODO: Import foundation components after implementing foundation_sar.py")
print("✅ Once imported, you can create sample cases for agent testing")

📋 TODO: Import foundation components after implementing foundation_sar.py
✅ Once imported, you can create sample cases for agent testing


In [4]:
# TODO: Test foundation components
# Once you've implemented foundation_sar.py, use this cell to:
# 1. Load CSV data
# 2. Create a DataLoader instance
# 3. Generate a sample case for testing agents

print("📋 Testing foundation components")
print("1. Load CSV data from ../data/")
print("2. Create ExplainabilityLogger instance")
print("3. Create DataLoader instance")
print("4. Generate sample case for agent testing\n")

# --- 1) Ensure output folder exists ---
audit_log_path = "../outputs/audit_logs/agent_development.jsonl"
os.makedirs(os.path.dirname(audit_log_path), exist_ok=True)

# --- 2) Create logger and loader ---
logger = ExplainabilityLogger(audit_log_path)
loader = DataLoader(logger)

# --- 3) Load CSVs ---
customers_df, accounts_df, transactions_df = load_csv_data("../data/")

print("✅ CSVs loaded")
print(f"   customers:     {customers_df.shape}")
print(f"   accounts:      {accounts_df.shape}")
print(f"   transactions:  {transactions_df.shape}\n")

# --- 4) Generate a sample case (pick first customer with at least 1 account + 1 txn) ---
sample_case = None

# Index for faster filtering
accounts_by_customer = accounts_df.groupby("customer_id")
txns_by_account = transactions_df.groupby("account_id")

for _, cust_row in customers_df.iterrows():
    cust_id = cust_row["customer_id"]

    if cust_id not in accounts_by_customer.groups:
        continue

    cust_accounts_df = accounts_by_customer.get_group(cust_id)
    account_ids = set(cust_accounts_df["account_id"].tolist())

    # Gather txns for those accounts
    cust_txns_list = []
    for acc_id in account_ids:
        if acc_id in txns_by_account.groups:
            cust_txns_list.extend(txns_by_account.get_group(acc_id).to_dict(orient="records"))

    # CaseData validator requires non-empty transactions
    if not cust_txns_list:
        continue

    # Build dict payloads for DataLoader
    customer_data = cust_row.to_dict()
    if "ssn_last_4" in customer_data:
        customer_data["ssn_last_4"] = str(customer_data["ssn_last_4"]).zfill(4)

    account_data = cust_accounts_df.to_dict(orient="records")

    # Normalize transaction dicts to satisfy TransactionData Optional[str] fields
    cust_txns_list = [normalize_transaction_dict(t) for t in cust_txns_list]

    # Create the case via DataLoader (logs an audit entry)
    sample_case = loader.create_case_from_data(
        customer_data=customer_data,
        account_data=account_data,
        transaction_data=cust_txns_list,
    )
    break

if sample_case is None:
    raise RuntimeError(
        "No valid sample case found. Ensure your CSVs contain at least one customer with "
        "an account and at least one transaction linked to that account."
    )

print("✅ Sample case created")
print(f"   case_id: {sample_case.case_id}")
print(f"   customer_id: {sample_case.customer.customer_id}")
print(f"   accounts: {len(sample_case.accounts)}")
print(f"   transactions: {len(sample_case.transactions)}")

# --- 5) Quick audit log validation ---
print("\n🧾 Audit log status")
print(f"   entries_in_memory: {len(logger.entries)}")
print(f"   last_entry_agent_type: {logger.entries[-1].get('agent_type') if logger.entries else None}")
print(f"   log_file_written: {os.path.exists(audit_log_path)}")

# sample_case is now ready to be used by agents in later cells


📋 Testing foundation components
1. Load CSV data from ../data/
2. Create ExplainabilityLogger instance
3. Create DataLoader instance
4. Generate sample case for agent testing

✅ CSVs loaded
   customers:     (150, 10)
   accounts:      (178, 7)
   transactions:  (4268, 9)

✅ Sample case created
   case_id: bb963997-7f9a-41e1-8246-21012231ac35
   customer_id: CUST_0002
   accounts: 1
   transactions: 11

🧾 Audit log status
   entries_in_memory: 1
   last_entry_agent_type: DataLoader
   log_file_written: True


## 🔍 Phase 2: Risk Analyst Agent Development

The Risk Analyst Agent uses **Chain-of-Thought prompting** to systematically analyze suspicious activity patterns.

### 📚 Understanding Chain-of-Thought Prompting

Chain-of-Thought (CoT) prompting guides AI models through step-by-step reasoning:

1. **Explicit Steps**: Break complex reasoning into clear phases
2. **Sequential Logic**: Each step builds on previous ones
3. **Domain Expertise**: Frame AI as subject matter expert
4. **Structured Output**: Guide toward specific response format

In [5]:
# TODO: Test Chain-of-Thought prompt design
# This cell helps you design and test your CoT prompt structure

def design_cot_prompt():
    """Design and test Chain-of-Thought prompt for risk analysis"""
    
    system_prompt = """
    You are a Senior Financial Crime Risk Analyst specializing in Suspicious Activity Report (SAR) triage.

    Use Chain-of-Thought (step-by-step reasoning) internally to analyze the case in five phases:
    1) Data Review: Summarize customer profile, accounts, and transactions.
    2) Pattern Recognition: Identify unusual behavior and suspicious patterns (amounts, frequency, channels, counterparties, geographies).
    3) Regulatory Mapping: Map observed indicators to common AML typologies and red flags.
    4) Risk Quantification: Assign a risk_level (Low|Medium|High|Critical) and a confidence_score (0.0–1.0) based on evidence strength.
    5) Classification Decision: Choose one primary classification and justify it concisely.

    Classification categories (choose exactly ONE):
    - Structuring
    - Sanctions
    - Fraud
    - Money_Laundering
    - Other

    Output requirements:
    - Return ONLY valid JSON (no markdown, no extra text).
    - Use exactly the following schema:

    {
    "classification": "Structuring|Sanctions|Fraud|Money_Laundering|Other",
    "confidence_score": 0.0-1.0,
    "reasoning": "Concise rationale referencing the key facts and indicators",
    "key_indicators": ["indicator1", "indicator2", "indicator3"],
    "risk_level": "Low|Medium|High|Critical"
    }
    """
    
    return system_prompt

# Test your prompt design
cot_prompt = design_cot_prompt()
print("🧠 Chain-of-Thought Prompt Design:")
print(cot_prompt[:200] + "...")
print("\n📋 Next step: Ensure the same prompt (or equivalent) is set in risk_analyst_agent.py as agent.system_prompt")

🧠 Chain-of-Thought Prompt Design:

    You are a Senior Financial Crime Risk Analyst specializing in Suspicious Activity Report (SAR) triage.

    Use Chain-of-Thought (step-by-step reasoning) internally to analyze the case in five ph...

📋 Next step: Ensure the same prompt (or equivalent) is set in risk_analyst_agent.py as agent.system_prompt


In [6]:
# TODO: Implement and test Risk Analyst Agent - SIMPLE SMOKE TEST
# Students: Write a basic smoke test to verify your agent works

def simple_risk_analyst_smoke_test():
    # ---- Fake OpenAI client (prevents external calls) ----
    class _FakeChoiceMsg:
        def __init__(self, content: str):
            self.content = content

    class _FakeChoice:
        def __init__(self, content: str):
            self.message = _FakeChoiceMsg(content)

    class _FakeResponse:
        def __init__(self, content: str):
            self.choices = [_FakeChoice(content)]

    class _FakeChatCompletions:
        def create(self, model, temperature, max_tokens, messages):
            output = RiskAnalystOutput(
                classification="Structuring",
                confidence_score=0.82,
                reasoning="Repeated cash deposits close to reporting thresholds indicate possible structuring.",
                key_indicators=[
                    "near-threshold cash deposits",
                    "repeated deposits"
                ],
                risk_level="High",
            )

            return _FakeResponse(
                json.dumps(output.model_dump())
            )

    class _FakeChat:
        def __init__(self):
            self.completions = _FakeChatCompletions()

    class FakeOpenAIClient:
        def __init__(self):
            self.chat = _FakeChat()

    # ---- Create VALID test data (must satisfy foundation_sar validators) ----
    customer = CustomerData(
        customer_id="CUST_0001",
        name="John Doe",
        date_of_birth="1985-04-12",
        ssn_last_4="1234",
        address="123 Main St, Miami, FL",
        customer_since="2018-06-01",
        risk_rating="Medium",
    )

    account = AccountData(
        account_id="CUST_0001_ACC_1",
        customer_id="CUST_0001",
        account_type="Checking",
        opening_date="2018-06-01",
        current_balance=15250.75,
        average_monthly_balance=12000.50,
        status="Active",
    )

    txn = TransactionData(
        transaction_id="TXN_0001",
        account_id="CUST_0001_ACC_1",  # must match account_id above
        transaction_date="2025-01-01",
        transaction_type="Cash_Deposit",
        amount=9900.00,
        description="Cash deposit at branch",
        method="Branch",
        location="Miami, FL",
        counterparty=None,
    )

    case = CaseData(
        case_id="CASE_SMOKE_0001",
        customer=customer,
        accounts=[account],
        transactions=[txn],  # cannot be empty
        case_created_at=datetime.now().isoformat(),
        data_sources={"test": "smoke"},
    )

    # ---- Initialize agent + logger ----
    audit_path = "outputs_smoke_test.jsonl"
    logger = ExplainabilityLogger(audit_path)
    client = FakeOpenAIClient()
    agent = RiskAnalystAgent(client, logger, model="gpt-4")

    # ---- Run + validate ----
    try:
        result = agent.analyze_case(case)

        # Basic structure checks
        assert result is not None
        assert hasattr(result, "classification")
        assert hasattr(result, "confidence_score")
        assert hasattr(result, "reasoning")
        assert hasattr(result, "key_indicators")
        assert hasattr(result, "risk_level")

        assert result.classification in ["Structuring", "Sanctions", "Fraud", "Money_Laundering", "Other"]
        assert 0.0 <= float(result.confidence_score) <= 1.0
        assert isinstance(result.reasoning, str) and len(result.reasoning) > 0
        assert isinstance(result.key_indicators, list)
        assert result.risk_level in ["Low", "Medium", "High", "Critical"]

        # Logging checks
        assert len(logger.entries) >= 1
        assert logger.entries[-1]["agent_type"] == "RiskAnalyst"
        assert logger.entries[-1]["success"] is True

        print(f"✅ SUCCESS: classification='{result.classification}', confidence={result.confidence_score}")

    except Exception as e:
        print(f"❌ FAILED: {e}")
        raise
    finally:
        # Optional cleanup (keep file if you want evidence)
        if os.path.exists(audit_path):
            os.remove(audit_path)

# Run the smoke test
simple_risk_analyst_smoke_test()

✅ SUCCESS: classification='Structuring', confidence=0.82


### 🧪 Risk Analyst Testing Framework

In [7]:
# COMPREHENSIVE Risk Analyst Testing - Import Pre-Built Test Suite
# Students: Use our comprehensive test suite instead of writing your own

import pytest

# Notebook normalmente em: starter/notebooks
NOTEBOOK_DIR = os.getcwd()

# Caminho para: starter (raiz do projeto dentro de starter/)
PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, ".."))

# Caminho para: starter/tests
tests_path = os.path.join(PROJECT_ROOT, "tests")

if tests_path not in sys.path:
    sys.path.insert(0, tests_path)

print(f"📁 Added tests directory to Python path: {tests_path}")

def run_comprehensive_risk_analyst_tests():
    """
    Use pre-built comprehensive test suite to validate your Risk Analyst Agent

    These tests validate:
    - Agent initialization and configuration
    - Case analysis with valid inputs
    - JSON parsing and error handling
    - System prompt structure and content
    - API call parameters and responses
    - Helper method functionality
    """
    print("🧪 Comprehensive Risk Analyst Testing (TIER 2)")

    test_file = os.path.join(tests_path, "test_risk_analyst.py")
    if not os.path.exists(test_file):
        raise FileNotFoundError(f"Test file not found: {test_file}")

    print(f"🔍 Running test suite: {test_file}")

    result = pytest.main([
        test_file,
        "-v",
        "--tb=short",
        "-p", "no:cacheprovider",
        "-W", "ignore::pytest.PytestAssertRewriteWarning",
    ])

    if result == 0:
        print("✅ All Risk Analyst tests passed!")
    else:
        raise RuntimeError("❌ Some tests failed. Check pytest output above.")

# Quick preview of available tests
try:
    from test_risk_analyst import TestRiskAnalystAgent

    test_methods = [m for m in dir(TestRiskAnalystAgent) if m.startswith("test_")]

    print("\n📊 Preview of Comprehensive Risk Analyst Tests:")
    for method_name in test_methods[:5]:
        method = getattr(TestRiskAnalystAgent, method_name)
        doc = method.__doc__ or method_name.replace("_", " ").title()
        print(f"   • {doc}")

    if len(test_methods) > 5:
        print(f"   ... and {len(test_methods) - 5} more tests")

    print("\n💡 These tests validate edge cases you might not think of!")
    print("💡 Much more thorough than manual testing!")

except Exception as e:
    print(f"ℹ️ Test suite preview not available yet: {e}")

# Run comprehensive tests
run_comprehensive_risk_analyst_tests()

📁 Added tests directory to Python path: c:\Projects\Agentic AI For Financial Services\1. Prompting for LLM Reasoning and Planning for Financial Services\Project - Automated SAR Detection & Reporting System\starter\tests

📊 Preview of Comprehensive Risk Analyst Tests:
   • Test RiskAnalystAgent initializes properly
   • Test handling of invalid JSON response
   • Test successful case analysis with valid response
   • Test OpenAI API call uses correct parameters
   • Test handling of empty LLM response
   ... and 5 more tests

💡 These tests validate edge cases you might not think of!
💡 Much more thorough than manual testing!
🧪 Comprehensive Risk Analyst Testing (TIER 2)
🔍 Running test suite: c:\Projects\Agentic AI For Financial Services\1. Prompting for LLM Reasoning and Planning for Financial Services\Project - Automated SAR Detection & Reporting System\starter\tests\test_risk_analyst.py
============================= test session starts =============================
platform win32 -- Py

## ✅ Phase 3: Compliance Officer Agent Development

The Compliance Officer Agent uses **ReACT prompting** to generate regulatory-compliant SAR narratives.

### 📚 Understanding ReACT Prompting

ReACT (Reasoning + Action) prompting separates thinking and doing:

1. **Reasoning Phase**: Analyze situation and plan approach
2. **Action Phase**: Execute specific task with informed decisions
3. **Structured Workflow**: Consistent approach to complex tasks
4. **Regulatory Compliance**: Emphasis on meeting specific requirements

In [8]:
def design_react_prompt():
    """Design and test ReACT prompt for compliance narratives"""
    
    system_prompt = """
    You are a Senior Compliance Officer specializing in BSA/AML regulations.
    
    Your goal is to write a SAR (Suspicious Activity Report) narrative.
    
    USE THE ReACT FRAMEWORK:
    
    1. **REASONING (Thought Process):**
       - Review the Risk Analyst's findings (structuring, laundering, etc.).
       - Check applicable laws (e.g., Bank Secrecy Act, Patriot Act).
       - Plan a narrative that answers: Who, What, When, Where, Why.
    
    2. **ACTION (Output Generation):**
       - Write a concise narrative strictly under **120 words**.
       - Use professional, regulatory language (e.g., "no apparent lawful purpose").
       - Cite specific regulations.
    
    OUTPUT FORMAT (JSON ONLY):
    {
       "narrative": "Text...",
       "narrative_reasoning": "Explanation...",
       "regulatory_citations": ["List..."],
       "completeness_check": true
    }
    """
    
    return system_prompt

# Test your prompt design
react_prompt = design_react_prompt()
print("⚡ ReACT Prompt Design:")
print(react_prompt[:200] + "...")
print("\n📋 TODO: Complete the prompt in compliance_officer_agent.py")

⚡ ReACT Prompt Design:

    You are a Senior Compliance Officer specializing in BSA/AML regulations.
    
    Your goal is to write a SAR (Suspicious Activity Report) narrative.
    
    USE THE ReACT FRAMEWORK:
    
    1....

📋 TODO: Complete the prompt in compliance_officer_agent.py


In [9]:
from unittest.mock import Mock

from compliance_officer_agent import ComplianceOfficerAgent

def simple_compliance_officer_smoke_test():
    """
    STUDENT TODO: Write a simple smoke test for your Compliance Officer Agent
    """
    print("✅ Compliance Officer Smoke Test")
    
    try:
        mock_client = Mock()
        mock_response = Mock()
        mock_choice = Mock()
        mock_message = Mock()

        fake_llm_json = """
        {
            "narrative": "Customer Smoke Test User (CUST_SMOKE) conducted a cash deposit of $9,900 on 2024-01-01 via 'Cash' method. This transaction appears designed to evade the $10,000 Currency Transaction Report (CTR) filing threshold, a pattern indicative of structuring under the Bank Secrecy Act.",
            "narrative_reasoning": "Detected explicit structuring pattern just below threshold.",
            "regulatory_citations": ["31 CFR 1020.320", "31 USC 5324"],
            "completeness_check": true
        }
        """

        mock_message.content = fake_llm_json
        mock_choice.message = mock_message
        mock_response.choices = [mock_choice]
        

        mock_client.chat.completions.create.return_value = mock_response

        # 1. Initialize Agent with Mock Configuration
        
        mock_logger = Mock() 

        logger_to_use = logger if 'explainability_logger' in globals() else mock_logger
        
        agent = ComplianceOfficerAgent(mock_client, logger_to_use)
        
        # 2. Create Dummy Data
        dummy_customer = CustomerData(
            customer_id="CUST_SMOKE", name="Smoke Test User", 
            date_of_birth="1990-01-01", ssn_last_4="1234", 
            address="123 Test Ln", customer_since="2020-01-01", risk_rating="Medium"
        )

        dummy_account = AccountData(
            account_id="ACC_SMOKE", customer_id="CUST_SMOKE", 
            account_type="Checking", opening_date="2020-01-01", 
            current_balance=5000.0, average_monthly_balance=1000.0, status="Active"
        )

        dummy_txn = TransactionData(
            transaction_id="TXN_1", account_id="ACC_SMOKE", 
            amount=9900.0, transaction_type="Cash_Deposit", 
            transaction_date="2024-01-01", 
            description="Cash", method="Cash"
        )

        dummy_case = CaseData(
            case_id="CASE_SMOKE", 
            customer=dummy_customer, 
            accounts=[dummy_account], 
            transactions=[dummy_txn], 
            case_created_at="2024-01-01T12:00:00Z", 
            data_sources={"test_source": "manual_entry"}
        )
        
        dummy_risk = RiskAnalystOutput(
            classification="Structuring", confidence_score=0.9, 
            reasoning="Just below threshold", key_indicators=["threshold"], 
            risk_level="High"
        )

        # 3. Call Generation
        print("⏳ Generating narrative...")
        result = agent.generate_compliance_narrative(dummy_case, dummy_risk)
        
        # 4. Verify Results
        word_count = len(result.narrative.split())
        print(f"📄 Narrative generated ({word_count} words)")
        print(f"📝 Preview: {result.narrative[:100]}...")
        
        if word_count <= 120 and result.completeness_check:
            print("✅ SUCCESS: Smoke test passed!")
        else:
            print(f"❌ FAILED: Word count {word_count} > 120 or completeness check failed")

    except Exception as e:
        print(f"❌ FAILED: Exception occurred - {type(e).__name__}: {e}")
        # import traceback
        # traceback.print_exc()

simple_compliance_officer_smoke_test()

✅ Compliance Officer Smoke Test
⏳ Generating narrative...
🔍 DEBUG: Starting narrative generation...
📄 Narrative generated (40 words)
📝 Preview: Customer Smoke Test User (CUST_SMOKE) conducted a cash deposit of $9,900 on 2024-01-01 via 'Cash' me...
✅ SUCCESS: Smoke test passed!


### 🧪 Compliance Officer Testing Framework

In [10]:
# COMPREHENSIVE Compliance Officer Testing - Import Pre-Built Test Suite
# Students: Use our comprehensive test suite instead of writing your own

import pytest
import os
import sys

NOTEBOOK_DIR = os.getcwd()

PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, ".."))

tests_path = os.path.join(PROJECT_ROOT, "tests")

if tests_path not in sys.path:
    sys.path.insert(0, tests_path)

print(f"📁 Added tests directory to Python path: {tests_path}")

def run_comprehensive_compliance_officer_tests():
    """
    Use pre-built comprehensive test suite to validate your Compliance Officer Agent
    """
    print("🧪 Comprehensive Compliance Officer Testing (TIER 2)")

    test_file = os.path.join(tests_path, "test_compliance_officer.py")
    
    if not os.path.exists(test_file):
        raise FileNotFoundError(f"Test file not found: {test_file}")

    print(f"🔍 Running test suite: {test_file}")

    result = pytest.main([
        test_file,
        "-v",
        "--tb=short",
        "-p", "no:cacheprovider",
        "-W", "ignore::pytest.PytestAssertRewriteWarning",
    ])

    if result == 0:
        print("✅ All Compliance Officer tests passed!")
    else:
        print("❌ Some tests failed. Check the output above for details.")

# Quick preview of available tests
try:
    from test_compliance_officer import TestComplianceOfficerAgent
    
    # Get all test methods
    test_methods = [method for method in dir(TestComplianceOfficerAgent) 
                    if method.startswith('test_')]
    
    print("\n📝 Preview of Comprehensive Compliance Officer Tests:")
    for method_name in test_methods[:5]:  # Show first 5
        method = getattr(TestComplianceOfficerAgent, method_name)
        doc = method.__doc__ or method_name.replace('_', ' ').title()
        print(f"   • {doc}")
    if len(test_methods) > 5:
        print(f"   ... and {len(test_methods) - 5} more tests")
        
    print("\n💡 These tests validate regulatory compliance requirements!")
    print("💡 Includes word limits, citations, and required elements!")
    
except Exception as e:
    print(f"ℹ️ Preview not available (this is ok if tests run): {e}")

# Call the function
run_comprehensive_compliance_officer_tests()

📁 Added tests directory to Python path: c:\Projects\Agentic AI For Financial Services\1. Prompting for LLM Reasoning and Planning for Financial Services\Project - Automated SAR Detection & Reporting System\starter\tests

📝 Preview of Comprehensive Compliance Officer Tests:
   • Test ComplianceOfficerAgent initializes properly
   • Test OpenAI API call uses correct parameters
   • Test handling of empty LLM response
   • Test JSON extraction from code blocks
   • Test JSON extraction from plain text response
   ... and 5 more tests

💡 These tests validate regulatory compliance requirements!
💡 Includes word limits, citations, and required elements!
🧪 Comprehensive Compliance Officer Testing (TIER 2)
🔍 Running test suite: c:\Projects\Agentic AI For Financial Services\1. Prompting for LLM Reasoning and Planning for Financial Services\Project - Automated SAR Detection & Reporting System\starter\tests\test_compliance_officer.py
============================= test session starts ==============

In [11]:
# COMPLETE AGENT TESTING - Two-Tier Approach
# Students: Use this to test both agents together

def complete_agent_testing_workflow():
    """
    Complete testing workflow using two-tier approach:
    
    TIER 1: Simple Smoke Tests (You write these)
    - Basic functionality verification
    - Quick sanity checks
    - Development debugging
    
    TIER 2: Comprehensive Test Suites (Pre-built for you)
    - Complex edge cases
    - Regulatory compliance validation
    - Professional-grade testing
    """
    print("🔬 Complete Agent Testing Workflow")
    print("=" * 50)
    
    print("\n📋 TIER 1: Simple Smoke Tests (DO FIRST)")
    print("   1. Write simple_risk_analyst_smoke_test() - verify basic functionality")
    print("   2. Write simple_compliance_officer_smoke_test() - verify basic functionality")
    print("   3. Fix any basic issues before moving to Tier 2")
    
    print("\n🧪 TIER 2: Comprehensive Test Suites (DO AFTER TIER 1 PASSES)")
    print("   1. Run comprehensive risk analyst test suite (10 comprehensive tests)")
    print("   2. Run comprehensive compliance officer test suite (10 comprehensive tests)")
    print("   3. Get detailed pass/fail results with specific feedback")
    
    print("\n💡 WHY THIS APPROACH?")
    print("   ✅ Tier 1: Quick feedback while developing")
    print("   ✅ Tier 2: Professional validation without writing complex tests")
    print("   ✅ Saves time: You focus on implementation, not test creation")
    print("   ✅ Better coverage: Our test suites test edge cases you might miss")

# Quick test runner when both agents are ready
def run_both_agents_quick_test():
    """Quick test of both agents using pre-built test suites"""
    print("🚀 Quick Test of Both Agents")
    print("📋 TODO: Uncomment when both agents are implemented")
    
    # Uncomment when ready:
    try:
        
        print("🔍 Running quick tests for both agents...")
        
        pytest_flags = [
            "-v", 
            "--tb=short",
            "-p", "no:cacheprovider",
            "-W", "ignore::pytest.PytestAssertRewriteWarning"
        ]

        # Run a subset of tests for quick validation
        risk_result = pytest.main([
            f"{tests_path}/test_risk_analyst.py::TestRiskAnalystAgent::test_agent_initialization",
            f"{tests_path}/test_risk_analyst.py::TestRiskAnalystAgent::test_analyze_case_success"
        ] + pytest_flags)
        
        compliance_result = pytest.main([
            f"{tests_path}/test_compliance_officer.py::TestComplianceOfficerAgent::test_agent_initialization", 
            f"{tests_path}/test_compliance_officer.py::TestComplianceOfficerAgent::test_generate_compliance_narrative_success"
        ] + pytest_flags)
        
        if risk_result == 0 and compliance_result == 0:
            print("🎉 Both agents working! Ready for full test suite testing!")
        else:
            print("⚠️ Fix issues before running comprehensive tests")
            if risk_result != 0:
                print("   🔍 Risk Analyst needs fixes")
            if compliance_result != 0:
                print("   📝 Compliance Officer needs fixes")
                
    except ImportError as e:
        print(f"❌ Import Error: {e}")
        print("💡 Make sure both agents are implemented")

complete_agent_testing_workflow()
run_both_agents_quick_test()

🔬 Complete Agent Testing Workflow

📋 TIER 1: Simple Smoke Tests (DO FIRST)
   1. Write simple_risk_analyst_smoke_test() - verify basic functionality
   2. Write simple_compliance_officer_smoke_test() - verify basic functionality
   3. Fix any basic issues before moving to Tier 2

🧪 TIER 2: Comprehensive Test Suites (DO AFTER TIER 1 PASSES)
   1. Run comprehensive risk analyst test suite (10 comprehensive tests)
   2. Run comprehensive compliance officer test suite (10 comprehensive tests)
   3. Get detailed pass/fail results with specific feedback

💡 WHY THIS APPROACH?
   ✅ Tier 1: Quick feedback while developing
   ✅ Tier 2: Professional validation without writing complex tests
   ✅ Saves time: You focus on implementation, not test creation
   ✅ Better coverage: Our test suites test edge cases you might miss
🚀 Quick Test of Both Agents
📋 TODO: Uncomment when both agents are implemented
🔍 Running quick tests for both agents...
============================= test session starts =========

## 🔗 Phase 4 Preview: Agent Integration

Once both agents are working, you'll integrate them into a complete workflow.

In [12]:
# TODO: Preview of integrated workflow
# This will be fully implemented in the next notebook

def preview_integrated_workflow():
    """Preview of how agents will work together"""
    
    workflow_steps = [
        "1. 📊 Load and validate case data",
        "2. 🔍 Risk Analyst performs Chain-of-Thought analysis",
        "3. 👤 Human review and approval gate",
        "4. ✅ Compliance Officer generates ReACT narrative (if approved)",
        "5. 📄 Generate complete SAR document",
        "6. 📊 Log audit trail and efficiency metrics"
    ]
    
    print("🔗 Integrated SAR Processing Workflow:")
    for step in workflow_steps:
        print(step)
    
    print("\n💡 Key Benefits:")
    print("• Two-stage processing reduces AI costs")
    print("• Human oversight ensures regulatory compliance")
    print("• Complete audit trails for examination")
    print("• Standardized analytical approaches")

preview_integrated_workflow()

🔗 Integrated SAR Processing Workflow:
1. 📊 Load and validate case data
2. 🔍 Risk Analyst performs Chain-of-Thought analysis
3. 👤 Human review and approval gate
4. ✅ Compliance Officer generates ReACT narrative (if approved)
5. 📄 Generate complete SAR document
6. 📊 Log audit trail and efficiency metrics

💡 Key Benefits:
• Two-stage processing reduces AI costs
• Human oversight ensures regulatory compliance
• Complete audit trails for examination
• Standardized analytical approaches


## 📝 Development Checklist - Two-Tier Testing Approach

### ✅ Risk Analyst Agent (Phase 2)
- [X] Implement Chain-of-Thought system prompt
- [X] Create `analyze_case` method with error handling
- [X] Add JSON parsing and validation
- [X] **TIER 1**: Write simple smoke test (verify basic functionality)
- [X] **TIER 2**: Run comprehensive pre-built test suite (10 comprehensive tests)
- [X] Fix any issues identified by test suite

### ✅ Compliance Officer Agent (Phase 3)  
- [X] Implement ReACT system prompt
- [X] Create `generate_compliance_narrative` method
- [X] Add narrative validation (word count, terminology)
- [X] **TIER 1**: Write simple smoke test (verify basic functionality)
- [X] **TIER 2**: Run comprehensive pre-built test suite (10 comprehensive tests)
- [X] Fix any issues identified by test suite

### ✅ Testing Strategy Benefits
- [X] **Time Savings**: Focus on implementation, not complex test creation
- [X] **Better Coverage**: Pre-built test suites test edge cases you might miss
- [X] **Quick Feedback**: Simple smoke tests for rapid development cycles
- [X] **Professional Validation**: Comprehensive test suites ensure production readiness
- [X] **Regulatory Compliance**: Built-in checks for SAR requirements

### 💡 **Testing Workflow**
1. **Start with Tier 1**: Write simple smoke tests to verify your agents don't crash
2. **Fix basic issues**: Iterate quickly with simple tests during development
3. **Move to Tier 2**: Run comprehensive test suites when basic functionality works
4. **Analyze results**: Use detailed feedback to improve agent performance
5. **Iterate**: Refine prompts and logic based on test results

## 🚀 Next Steps

1. **Complete Agent Implementation**: Finish both agent classes in the src/ directory
2. **Run Two-Tier Testing**: Start with smoke tests, then comprehensive test suites
3. **Workflow Integration**: Move to the next notebook for complete system integration
4. **Human-in-the-Loop**: Implement decision gates and review processes

## 📊 Available Test Suites Summary

**Risk Analyst Test Suite (10 tests):**
- Agent initialization and configuration
- Case analysis with valid JSON responses
- JSON parsing and error handling
- System prompt structure validation
- API call parameter verification
- Helper method functionality
- Edge case handling

**Compliance Officer Test Suite (10 tests):**
- Agent initialization and configuration
- Narrative generation with valid responses
- Word count validation (≤120 words)
- Regulatory citations inclusion
- JSON parsing and error handling
- ReACT prompt structure validation
- API call parameter verification

**Ready to build intelligent agents with professional-grade testing! 🕵️‍♀️**